In [1]:
#import the necessary modules 
%matplotlib inline 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
#import scipy
import sklearn
import itertools as it
from itertools import cycle 
import os.path as op
import timeit 
import json
from matplotlib import animation
import matplotlib.font_manager as font_manager
from collections import namedtuple
#from functools import partial
#from pathlib import Path

In [2]:
# Set plotting style
plt.style.use('seaborn-white')

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
#import matplotlib.pyplot as plt

In [3]:
%matplotlib widget

In [4]:
import multiprocessing as m_proc
m_proc.cpu_count()

4

### Now use MD Analysis to calculate no. of frames a center PEG residues and terminal PEG residue is with 4 Angstroms of BSA (0.9 nm restrained system)

Import MDAnalysis

In [5]:
from prot_polymer_analysis import get_protresd_list, aa_frmcount, grptwocnt_aa, gtwo_trjcnt 
from prot_polymer_analysis import frac_cont, bavg_frac_cnt, prot_poly_cntmovie, AA_list_org

In [6]:
# Import MDAnalysis
import MDAnalysis as mda
import MDAnalysis.analysis.distances as maa_dist

### First table will be total fractional contacts and oligomer occupancy values for each Rg value 

#### Distance-based analysis 

Find residues that have at least one atom within a cutoff $d = 4.0$ Angstrom near water molecules in BSA/water simulation

Calculate the number of surface bsa residues from a 1 ns BSA/water simulation

In [7]:
#Units of Angstroms 
dmax = 4.0 

In [8]:
def middle_of_band(band_start, band_stop, plot_min=0, plot_max=60):
    half_way = (band_stop - band_start) / 2
    mid_band = band_start + half_way
    plot_fraction = (mid_band - plot_min) / (plot_max - plot_min)

    return plot_fraction

# 0.9 nm PEG restrained Rg 100 ns trajectory

Load the rg = 1.2 nm (3 PLGA N = 20 oligomer/BSA system) 

In [9]:
# Set up the MD Simulation, Make sure you do gmx trjconv -s topol.tpr -f confout.gro  -o new_conf.pdb 
# -dump 0 -n bsaplga_nk.ndx to generate 
# a new pdb file that contains unique chain identifiers 
u_n20PEG = mda.Universe("0.9nmPEG_res/0.9nm_bsapegonly.pdb", "0.9nmPEG_res/nopbc_0.9pegbsa.xtc")

In [10]:
u_n20PEG

<Universe with 9665 atoms>

Check that we are on the first frame

In [12]:
u_n20PEG.trajectory.frame

0

In [13]:
pn20_len = len(u_n20PEG.trajectory)
pn20_len

10001

In [16]:
# Select one polymer chain, heavy atoms only 
#all_pn20 = u_pn20.select_atoms("(resname sPLG PLG tPLG and segid B) and not type H")

#Select all the PLGA residues, heavy atoms only
all_n20PEG = u_n20PEG.select_atoms("resname sPEG PEG tPEG and not type H")
#list(all_n20PEG)

In [18]:
# Select BSA residues, heavy atoms only 
prot_09nm = u_n20PEG.select_atoms("protein and not type H")
prot_09nm

<AtomGroup with 4653 atoms>

Calculate AA frame counts for PLGA residues, 1.2 nm RG restraint, 100ns trajectory 

In [19]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_len - 1
s_time = timeit.default_timer()
h2di_09nm = aa_frmcount(prot_09nm, all_n20PEG, dmax, u_n20PEG, start, end)
timeit.default_timer() - s_time

247.88068541139364

In [20]:
len(h2di_09nm.keys())

194

In [21]:
pr_res_PEG = list(prot_09nm.residues)
ss_res_PEG = [str(row) for row in pr_res_PEG]
rkg_n = {key:h2di_09nm[key][1] for key, value in h2di_09nm.items()}
plg_09nm_occ = pd.DataFrame(data=ss_res_PEG, columns=["BSA_des_res"])
plg_09nm_occ['mda_occ_0.9nm'] = plg_09nm_occ['BSA_des_res'].map(rkg_n)
plg_09nm_occ['mda_occ_0.9nm'] = plg_09nm_occ['mda_occ_0.9nm'].replace('nan', np.nan).fillna(0)
plg_09nm_occ['mda_occ_0.9nm'] = plg_09nm_occ['mda_occ_0.9nm'].round(2)
plg_09nm_occ

,BSA_des_res,mda_occ_0.9nm
0,"<Residue ASP, 1>",0.00
1,"<Residue THR, 2>",0.00
2,"<Residue HIS, 3>",0.00
3,"<Residue LYS, 4>",0.00
4,"<Residue SER, 5>",0.00
...,...,...
578,"<Residue GLN, 579>",0.01
579,"<Residue THR, 580>",0.07
580,"<Residue ALA, 581>",0.02
581,"<Residue LEU, 582>",0.00


In [22]:
plg_09nm_occ['mda_occ_0.9nm'][plg_09nm_occ['mda_occ_0.9nm'] != 0]

31     0.08
32     0.11
33     0.02
34     0.03
36     0.01
       ... 
575    0.18
576    0.11
578    0.01
579    0.07
580    0.02
Name: mda_occ_0.9nm, Length: 124, dtype: float64

In [23]:
bsa_r = np.array(list(prot_09nm.resids)) # shape is 4652
m_occ_09r = np.array(list(plg_09nm_occ['mda_occ_0.9nm'])) # shape is 583
m_occ = np.zeros(shape=(4653))
at_ind = np.where(bsa_r[:-1] != bsa_r[1:])[0]
at_in_nw = np.sort(np.append([0,4653],at_ind))
nw_v = 0
for i in range(583):
    b = at_in_nw[i+1] +1
    m_occ[nw_v:b] = m_occ_09r[i]
    nw_v = at_in_nw[i+1] + 1 

In [24]:
m_occ[633:646]

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.01])

### Visualize Occupanct on protein 

In [25]:
prot_09nm.occupancies = m_occ
prot_09nm.occupancies

array([0., 0., 0., ..., 0., 0., 0.])

In [26]:
with mda.Writer("prot_09nmpegRes.pdb") as pdb:
    pdb.write(prot_09nm)

In [ ]:
# Frame count and occupancy for each residue
#h2di

In [ ]:
len(h2di.keys())

### Residue Importance: 0.9 nm restrained 

In [27]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trjmap_09nmPEG = prot_poly_cntmovie(prot_09nm, all_n20PEG, dmax, u_n20PEG, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [28]:
np.save('0.9nmPEG_res.npy', trjmap_09nmPEG)    # .npy extension is added if not given

In [ ]:
trjmap_09nmPEG = np.load("1.2nm_res.npy", allow_pickle=True)

In [29]:
trjmap_09nmPEG[0].shape

(583, 60)

In [30]:
np.sum(trjmap_09nmPEG[1000][0])

0.0

In [31]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [32]:
pp_09nm_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        pp_09nm_ct[i][j] = np.sum(trjmap_09nmPEG[i][j])

In [33]:
pp_09nmtot = np.zeros(shape=(583))
for i in range(583):
    pp_09nmtot[i] = np.sum(pp_09nm_ct[:,i])
#pp_12nmtot

In [34]:
np.nonzero(pp_09nmtot)

(array([ 31,  32,  33,  34,  35,  36,  37,  57,  58,  59,  81,  82,  83,
         85,  99, 106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 123, 124, 127, 128, 131, 135, 139, 142, 143,
        169, 170, 171, 172, 173, 174, 175, 177, 178, 179, 181, 182, 184,
        185, 186, 201, 202, 203, 204, 205, 206, 207, 241, 277, 278, 279,
        280, 284, 311, 318, 319, 321, 349, 350, 352, 353, 354, 356, 357,
        358, 360, 361, 362, 363, 364, 365, 366, 367, 370, 374, 377, 378,
        381, 393, 394, 398, 399, 415, 417, 419, 430, 433, 434, 436, 437,
        443, 464, 465, 466, 467, 468, 469, 470, 472, 473, 474, 476, 477,
        478, 481, 484, 485, 488, 489, 490, 492, 493, 494, 495, 496, 497,
        498, 499, 500, 501, 502, 503, 504, 506, 507, 508, 509, 510, 511,
        512, 513, 514, 515, 516, 517, 518, 519, 520, 522, 523, 536, 544,
        545, 548, 550, 551, 554, 555, 556, 557, 558, 559, 560, 561, 562,
        563, 564, 565, 566, 567, 568, 569, 570, 571

In [37]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 2
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(y_pos, pp_09nmtot, align='center',width=wid, alpha=1, label='0.9 nm PEG')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,50000])
plt.legend(fontsize=14)
plt.ylabel(r'Total No. of PEG contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PEG oligomer residue within a 100 ns trajectory block

In [38]:
a_a = ["GLY","ALA","VAL","LEU","ILE","MET","PHE","TRP","PRO","SER","THR","CYS","TYR","ASN","GLN","ASP"
               ,"GLU","LYS","ARG","HIS"]

In [39]:
# This code chunk gets the BSA residues and their corresponding number in a pandas dataframe 
red_bsa = []
bh = np.arange(0,584)
for i in range(583):
    b_str = str(list(prot_09nm.residues[i:i+1]))
    if str(bh[i+1]) in b_str: 
        red_bsa.append(str(b_str[10:13])+" "+str(bh[i+1]))

In [41]:
pr_res = list(prot_09nm.residues)
ss_res = [str(row) for row in pr_res]
rkg = {key:h2di_09nm[key][0] for key, value in h2di_09nm.items()}
plg_09nmaa = pd.DataFrame(data=ss_res, columns=["BSA_des_res"])
plg_09nmaa['mda_plga_frm_0.9nm'] = plg_09nmaa['BSA_des_res'].map(rkg)
plg_09nmaa['BSA_des_res'] = red_bsa
plg_09nmaa['mda_plga_frm_0.9nm'] = plg_09nmaa['mda_plga_frm_0.9nm'].replace('nan', np.nan).fillna(0)
plg_09nmaa.tail()

,BSA_des_res,mda_plga_frm_0.9nm
578,GLN 579,107.0
579,THR 580,731.0
580,ALA 581,211.0
581,LEU 582,3.0
582,ALA 583,3.0


In [42]:
# Read in data from the oputput of wrapper.sh, where the frame count is given for each BSA residue that was within 
# 4 angstroms of PLGA trimer 
wat_data = pd.read_csv('occ_BSA1ns.txt', sep=" ", header=None, usecols=None ,index_col=None)
wat_data.columns = ["BSA_res_no","No. of frames (VMD)"]
wat_data = wat_data.drop("BSA_res_no", axis=1)

pr_res = list(prot_09nm.residues)
ss_res = [str(row) for row in pr_res]

wat_data['BSA_des_res'] = ss_res
wat_data = wat_data[['BSA_des_res',"No. of frames (VMD)"]]
#wat_data.head()

# load MDAnalysis values from MDA_BSA1ns.txt file(129003 atoms SOL group was used to calc. frame counts for txt.
# file)
h2ob_dict = json.load(open("MDA_BSA1ns.txt"))
wat_data['Mda_frames'] = wat_data['BSA_des_res'].map(h2ob_dict)


# From MD Analysis
#Get the count of bsa residues that have 1001 or 1002 frames ( I ran a 1 ns NPT simulation of 1 BSA in water )

#aa_count = pd.DataFrame(data=a_a)
c_list = []

for i in range(len(a_a)):
    count = 0
    for index, row in wat_data.iterrows():
        if a_a[i] in row["BSA_des_res"]:
            if row['Mda_frames'] == 1001: 
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames']))) 
            elif row['Mda_frames'] == 1000:
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames'])))
    c_list.append(str(str(a_a[i])+"  "+str(count)))
    
#c_list

# From VMD
#Get the count of bsa residues that have 1001 or 1002 frames ( I ran a 1 ns NPT simulation of 1 BSA in water )

#aa_count = pd.DataFrame(data=a_a)
vmd_list = []

for i in range(len(a_a)):
    count = 0
    for index, row in wat_data.iterrows():
        if a_a[i] in row["BSA_des_res"]:
            if row["No. of frames (VMD)"] == 1001: 
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames']))) 
            elif row["No. of frames (VMD)"] == 1002:
                count += 1
                #c_list.append(str(str(a_a[i])+"  "+str(row['No. of frames'])))
    vmd_list.append(str(str(a_a[i])+"  "+str(count)))

# Main difference is that Alanine 583 is counted for all 1001 frames. It seems VMD is unable to calc dist for that res
#vmd_list

#hydrophobic_res = ['ALA', 'ILE', 'LEU', 'VAL', 'GLY', 'PRO','PHE', 'TRP','MET']
#polar_res = ['ASN', 'CYS', 'GLN', 'SER', 'THR','TYR']
#neg_res = ['ASP', 'GLU']
#pos_res = ['ARG', 'HIS', 'LYS']
# aromatic_res = ['PHE', 'TRP', 'TYR', 'HIS']
#all_res = [pos_res, neg_res, polar_res, hydrophobic_res]

# Put the AA count in a pandas dataframe 
dg , ji = AA_list_org(c_list)
aa_count = pd.DataFrame(data=dg, index=None, columns=['Amino_acids'])
new_lf = pd.Series(data=ji, index=None)
vmg, vmdj = AA_list_org(vmd_list)
n2lf = pd.Series(data=vmdj, index=None)
aa_count['No_of_surf_res (MDAnalysis)'] = new_lf
aa_count['No_of_surf_res (VMD)'] = n2lf

In [43]:
apl_list = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plg_09nmaa.iterrows():
    if r_pl['mda_plga_frm_0.9nm'] != 0:
        apl_list.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_l = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_list)):
        if r_a['Amino_acids'] in apl_list[i]:
            count += 1
    cpl_l.append(count)      
        
aa_count['peg_0.9nm_100ns'] = cpl_l
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),peg_0.9nm_100ns
0,LYS,59,59,30
1,ARG,23,23,6
2,HIS,16,16,4
3,ASP,40,40,20
4,GLU,59,59,25
5,SER,23,23,6
6,THR,29,29,18
7,CYS,24,24,12
8,TYR,19,19,3
9,ASN,14,14,3


In [44]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['peg_0.9nm_100ns'].sum()

194

In [45]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [46]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['peg_0.9nm_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.39835728952772076

Calculate mean occupancy and the standard deviation for 1.2 nm trajectory

Numpy mean and std function was used to calculate mean occupancy and std dev using occ values from aa_frmcount output

In [48]:
# Mean occupancy and std deviation 
ll_mo = [value[1] for key, value in h2di_09nm.items()]
print("Mean Occpancy (1.2 nm Rg): "+str(np.mean(ll_mo)), "Occ. std. dev.: "+str(np.std(ll_mo)))

Mean Occpancy (1.2 nm Rg): 0.06283350515463916 Occ. std. dev.: 0.10240659289888122


### Calc. fractional contacts for each AA group type 

In [50]:
cd_12nm = frac_cont(h2di_09nm)
cd_12nm

{'Negative': [26902.0, 45, 115977.5111111111, 0.20873933170180392],
 'Positive': [26702.0, 40, 129504.70000000001, 0.2330859170132059],
 'Polar': [29110.0, 49, 115251.83673469388, 0.2074332441429725],
 'Hydrophobic': [39183.0, 60, 126691.7, 0.22802300667436762],
 'Aromatic': [4569.0, 13, 68183.53846153847, 0.12271850046765012]}

In [52]:
cd = frac_cont(h2di_09nm)
kklh = []
for key, value in cd.items():
    kklh.append(value[1])
# Must substract aromatic residues, since they are already counted
sum(kklh) - cd['Aromatic'][1]

194

In [ ]:
start = 0
end = pn20_len - 1
aa_12nm, l_f12nm = gtwo_trjcnt(prot, all_pn20, dmax, u_pn20, start, end)

In [ ]:
aa_12nm

In [ ]:
l_f12nm.shape

In [53]:
no_surf = aa_count['No_of_surf_res (MDAnalysis)'].sum()
no_surf

487

In [54]:
fcnt_rg09nm, prgrp_09nm, aa_matx09nm = bavg_frac_cnt(5, prot_09nm, all_n20PEG, dmax, u_n20PEG, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [55]:
fcnt_rg09nm

{'Negative': array([0.26978892, 0.24464354, 0.15986034, 0.2694437 , 0.17687541]),
 'Positive': array([0.04155673, 0.1394604 , 0.24358064, 0.2354513 , 0.30828733]),
 'Polar': array([0.22955145, 0.25647741, 0.23648116, 0.2351478 , 0.22533673]),
 'Hydrophobic': array([0.40567282, 0.34054284, 0.15364694, 0.14454155, 0.19553624]),
 'Aromatic': array([0.05343008, 0.0188758 , 0.20643091, 0.11541565, 0.09396429]),
 'total_frac': array([0.0698152 , 0.14373717, 0.15605749, 0.10472279, 0.25667351])}

In [56]:
prgrp_09nm

{'Negative': array([array([0.253     , 0.95445849]), array([3.4095    , 2.30169715]),
        array([7.4865    , 3.90266803]), array([7.892     , 3.42130618]),
        array([6.807     , 3.12518015])], dtype=object),
 'Positive': array([array([0.0215    , 0.16744477]), array([3.3615    , 2.44802323]),
        array([21.697     ,  8.73745907]),
        array([24.724     ,  5.44654239]),
        array([21.7855    ,  6.04404581])], dtype=object),
 'Polar': array([array([0.185     , 0.65174765]), array([3.3375    , 2.06702534]),
        array([10.666    ,  4.0496227]), array([8.5425    , 3.25717573]),
        array([8.898     , 3.84221238])], dtype=object),
 'Hydrophobic': array([array([0.516     , 1.74577891]), array([6.3125    , 3.48264896]),
        array([13.187     ,  5.40129901]), array([8.288     , 4.16473961]),
        array([8.3015    , 4.38378806])], dtype=object),
 'Aromatic': array([array([0.0105   , 0.1318702]), array([0.057     , 0.42278955]),
        array([2.057     , 2.398

In [57]:
fc_09nm_mean = np.array([np.mean(fcnt_rg09nm['Negative']), np.mean(fcnt_rg09nm['Positive'])
                        ,np.mean(fcnt_rg09nm['Polar']),np.mean(fcnt_rg09nm['Hydrophobic'])
                        , np.mean(fcnt_rg09nm['Aromatic'])])
fc_09nm_mean

array([0.22412238, 0.19366728, 0.23659891, 0.24798808, 0.09762335])

In [58]:
fc_09nm_std = np.array([np.std(fcnt_rg09nm['Negative']), np.std(fcnt_rg09nm['Positive'])
                       ,np.std(fcnt_rg09nm['Polar']),np.std(fcnt_rg09nm['Hydrophobic'])
                       , np.std(fcnt_rg09nm['Aromatic'])])
fc_09nm_std

array([0.04673855, 0.09322125, 0.0107124 , 0.10562527, 0.06375463])

In [59]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '500px'
fig.canvas.layout.height = '400px'
plt.bar(x_pos, fc_09nm_mean, yerr=fc_09nm_std, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts 0.9 nm Rg restrained', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20ns blocks

In [60]:
# Average of total fraction of contacts
np.mean(fcnt_rg09nm['total_frac'])

0.1462012320328542

In [61]:
# Std Deviation of total fraction of contacts
np.std(fcnt_rg09nm['total_frac'])

0.06302797728259481

### Avg no. PLGA residues per BSA AA residue group 

In [ ]:
prgrp_1_2nm

In [ ]:
mean_12nm = np.zeros(shape=5)
std_12nm = np.zeros(shape=5)
count = 0
for key, value in prgrp_1_2nm.items():
    mpl_12nm = []
    var_12nm = []
    for i in prgrp_1_2nm[str(key)].flat:
        mpl_12nm.append(i[0])
        var_12nm.append((i[1])**2)
    
    # calc frac cont averages
    mean_12nm[count] = np.mean(mpl_12nm)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_12nm[count] = np.std(mpl_12nm)
    # std_12nm[count] = np.sqrt(np.sum(var_12nm)/5)
    
    count += 1


In [ ]:
mean_12nm

In [ ]:
std_12nm 

In [ ]:
x_pos = np.arange(5)
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, mean_12nm, yerr=std_12nm, ecolor='black',capsize=5)
plt.title(r'No. of PLGA residues 1.2 nm Rg restrained', fontsize=15)
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

### Protein/polymer contact map movie

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 1.2 nm res", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.93, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_12nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.2nm_res.mp4',writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

In [ ]:
def com_plga_oligs(g2_atoms, n_mon):
    hj_1 = []
    hj_2 = []
    hj_3 = []
    for i in range(n_mon):
        if i == 0:
            absdif = np.abs(np.diff(np.equal(list(g2_atoms.resids),i+1).view(np.int8)))
            rep_pga = np.concatenate(([0], np.where(absdif == 1)[0]))
            rep_pga += 1
            #print(rep_pga)
            rpn20 = rep_pga.reshape(3,2)
            hj_1.append(g2_atoms[rpn20[0][0]:rpn20[0][1]].center_of_mass())
            hj_2.append(g2_atoms[rpn20[1][0]:rpn20[1][1]].center_of_mass())
            hj_3.append(g2_atoms[rpn20[2][0]:rpn20[2][1]].center_of_mass())
        elif i != 0 and i != 19:
            absdif = np.abs(np.diff(np.equal(list(g2_atoms.resids),i+1).view(np.int8)))
            rep_pga = np.where(absdif == 1)[0]+1
            #print(rep_pga)
            rpn20 = rep_pga.reshape(3,2)
            hj_1.append(g2_atoms[rpn20[0][0]:rpn20[0][1]].center_of_mass())
            hj_2.append(g2_atoms[rpn20[1][0]:rpn20[1][1]].center_of_mass())
            hj_3.append(g2_atoms[rpn20[2][0]:rpn20[2][1]].center_of_mass())
        elif i == 19: 
            absdif = np.abs(np.diff(np.equal(list(g2_atoms.resids),i+1).view(np.int8)))
            rep_pga = np.where(absdif == 1)[0]+1
            rep_pga = np.concatenate((rep_pga,[len(g2_atoms)]))
            #print(rep_pga)
            rpn20 = rep_pga.reshape(3,2)
            hj_1.append(g2_atoms[rpn20[0][0]:rpn20[0][1]].center_of_mass())
            hj_2.append(g2_atoms[rpn20[1][0]:rpn20[1][1]].center_of_mass())
            hj_3.append(g2_atoms[rpn20[2][0]:rpn20[2][1]].center_of_mass())
    oligs_cb = np.concatenate((np.array(hj_1), np.array(hj_2), np.array(hj_3)))
    
    return oligs_cb

In [ ]:
cm_12nmoligs = com_plga_oligs(all_pn20, 20)
#cm_12nmoligs

In [ ]:
len(cm_12nmoligs)

# 1.1 nm PEG restrained Rg 100 ns trajectory

Load the rg = 1.5 nm (3 PLGA N = 20 oligomer/BSA system) 

In [62]:
# Set up the MD Simulation
u11nm_n20PEG = mda.Universe("1.1nmPEG_res/1.1nm_bsapegonly.pdb", "1.1nmPEG_res/nopbc_1.1pegbsa.xtc")

In [63]:
u11nm_n20PEG

<Universe with 9665 atoms>

In [64]:
pn20_11nm = len(u11nm_n20PEG.trajectory)
pn20_11nm

10001

In [65]:
#Select all the PLGA residues, heavy atoms only 
n20PEG_11nm = u11nm_n20PEG.select_atoms("resname sPEG PEG tPEG and not type H")
n20PEG_11nm

<AtomGroup with 183 atoms>

In [66]:
# Select BSA residues, heavy atoms only 
prot_11nmPEG = u11nm_n20PEG.select_atoms("protein and not type H")
prot_11nmPEG

<AtomGroup with 4653 atoms>

### Contact Analysis

In [69]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_11nm - 1
s_time = timeit.default_timer()
h2di_11nm = aa_frmcount(prot_11nmPEG, n20PEG_11nm, dmax, u11nm_n20PEG, start, end)
timeit.default_timer() - s_time
#h2di_11nm

In [68]:
len(h2di_11nm.keys())

254

In [70]:
pr_res_PEG = list(prot_11nmPEG.residues)
ss_res_PEG = [str(row) for row in pr_res_PEG]
rkg_n = {key:h2di_11nm[key][1] for key, value in h2di_11nm.items()}
plg_09nm_occ['mda_occ_1.1nm'] = plg_09nm_occ['BSA_des_res'].map(rkg_n)
plg_09nm_occ['mda_occ_1.1nm'] = plg_09nm_occ['mda_occ_1.1nm'].replace('nan', np.nan).fillna(0)
plg_09nm_occ['mda_occ_1.1nm'] = plg_09nm_occ['mda_occ_1.1nm'].round(2)
plg_09nm_occ

,BSA_des_res,mda_occ_0.9nm,mda_occ_1.1nm
0,"<Residue ASP, 1>",0.00,0.00
1,"<Residue THR, 2>",0.00,0.00
2,"<Residue HIS, 3>",0.00,0.00
3,"<Residue LYS, 4>",0.00,0.00
4,"<Residue SER, 5>",0.00,0.00
...,...,...,...
578,"<Residue GLN, 579>",0.01,0.12
579,"<Residue THR, 580>",0.07,0.24
580,"<Residue ALA, 581>",0.02,0.14
581,"<Residue LEU, 582>",0.00,0.05


In [73]:
pr_res11nm = list(prot_11nmPEG.residues)
ss_res11nm = [str(row) for row in pr_res11nm]
rkg_11nm = {key:h2di_11nm[key][0] for key, value in h2di_11nm.items()}
plg_1_1nmaa = pd.DataFrame(data=ss_res11nm, columns=["BSA_des_res"])
plg_1_1nmaa['mda_plga_frm_1.1nm'] = plg_1_1nmaa['BSA_des_res'].map(rkg_11nm)
plg_1_1nmaa['BSA_des_res'] = red_bsa
plg_1_1nmaa['mda_plga_frm_1.1nm'] = plg_1_1nmaa['mda_plga_frm_1.1nm'].replace('nan', np.nan).fillna(0)
plg_1_1nmaa.head()

,BSA_des_res,mda_plga_frm_1.1nm
0,ASP 1,3.0
1,THR 2,0.0
2,HIS 3,0.0
3,LYS 4,2.0
4,SER 5,0.0


In [74]:
bsa_r = np.array(list(prot_11nmPEG.resids)) # shape is 4652
m_occ_11nm = np.array(list(plg_09nm_occ['mda_occ_1.1nm'])) # shape is 583
m_occ_11New = np.zeros(shape=(4653))
at_ind = np.where(bsa_r[:-1] != bsa_r[1:])[0]
at_in_nw = np.sort(np.append([0,4653],at_ind))
nw_v = 0
for i in range(583):
    b = at_in_nw[i+1] +1
    m_occ_11New[nw_v:b] = m_occ_11nm[i]
    nw_v = at_in_nw[i+1] + 1 

In [75]:
m_occ_11New[0:33]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
prot_11nmPEG.occupancies[3089:3099]

In [ ]:
list(prot_15nm.atoms[3089:3099])

In [76]:
np.nonzero(m_occ_11New)

(array([ 266,  267,  268, ..., 4650, 4651, 4652]),)

### Visualize Occupanct on protein 

In [77]:
prot_11nmPEG.occupancies = m_occ_11New
prot_11nmPEG.occupancies[0:33]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [78]:
with mda.Writer("prot_11nmpegRes.pdb") as pdb:
    pdb.write(prot_11nmPEG)

### Residue Importance: 1.1 nm restrained 

In [79]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trjmap_11nmPEG = prot_poly_cntmovie(prot_11nmPEG, n20PEG_11nm, dmax, u11nm_n20PEG, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [ ]:
trj_ppmap_15nm = np.load("1.5nm_res.npy", allow_pickle=True)

In [80]:
np.save('1.1nm_PEGres.npy', trjmap_11nmPEG)    # .npy extension is added if not given

In [81]:
trjmap_11nmPEG[0].shape

(583, 60)

In [82]:
np.sum(trjmap_11nmPEG[1000][0])

0.0

In [83]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [84]:
pp_11nm_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        pp_11nm_ct[i][j] = np.sum(trjmap_11nmPEG[i][j])

In [85]:
pp_11nmtot = np.zeros(shape=(583))
for i in range(583):
    pp_11nmtot[i] = np.sum(pp_11nm_ct[:,i])
#pp_12nmtot

In [86]:
np.nonzero(pp_11nmtot)

(array([  0,   3,  11,  31,  32,  33,  34,  37,  47,  50,  53,  54,  55,
         57,  77,  78,  80,  81,  82,  83,  84,  85,  88, 103, 104, 105,
        106, 108, 109, 110, 111, 112, 113, 114, 115, 116, 138, 142, 154,
        161, 164, 165, 166, 167, 168, 169, 171, 172, 174, 175, 177, 178,
        179, 181, 182, 184, 185, 203, 222, 223, 224, 225, 226, 227, 268,
        271, 272, 274, 275, 276, 280, 283, 286, 287, 290, 291, 292, 293,
        294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306,
        309, 310, 311, 312, 315, 316, 330, 334, 335, 336, 337, 338, 339,
        360, 361, 362, 363, 365, 366, 369, 370, 372, 376, 379, 380, 382,
        383, 386, 387, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399,
        400, 402, 403, 404, 407, 408, 415, 417, 418, 419, 420, 423, 430,
        433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445,
        457, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 472,
        476, 493, 494, 495, 496, 497, 498, 499, 500

In [94]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 1.5
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(y_pos+0.25, pp_11nmtot, align='center',width=wid, color='#562A8B', alpha=0.3, label='1.1 nm PEG')
plt.bar(y_pos, pp_09nmtot, align='center',width=wid, alpha=0.5, color='#1D77CF',label='0.9 nm PEG')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,70000])
plt.legend(fontsize=14)
plt.ylabel(r'Total No. of PEG contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PLGA oligomer residue within a 100 ns trajectory block

In [88]:
apl_15nm = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plg_1_1nmaa.iterrows():
    if r_pl['mda_plga_frm_1.1nm'] != 0:
        apl_15nm.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_15nm = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_15nm)):
        if r_a['Amino_acids'] in apl_15nm[i]:
            count += 1
    cpl_15nm.append(count)      
        
aa_count['peg_1.1nm_100ns'] = cpl_15nm
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),peg_0.9nm_100ns,peg_1.1nm_100ns
0,LYS,59,59,30,34
1,ARG,23,23,6,9
2,HIS,16,16,4,7
3,ASP,40,40,20,22
4,GLU,59,59,25,31
5,SER,23,23,6,10
6,THR,29,29,18,21
7,CYS,24,24,12,12
8,TYR,19,19,3,7
9,ASN,14,14,3,4


In [89]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['peg_1.1nm_100ns'].sum()

254

In [90]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [91]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['peg_1.1nm_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.5215605749486653

In [92]:
# Mean occupancy and std deviation 
ll_mo15 = [value[1] for key, value in h2di_11nm.items()]
print("Mean Occpancy (1.1 nm Rg): "+str(np.mean(ll_mo15)), "Occ. std. dev.: "+str(np.std(ll_mo15)))

Mean Occpancy (1.1 nm Rg): 0.08503031496062993 Occ. std. dev.: 0.1353175838184084


In [93]:
cd_11nm = frac_cont(h2di_11nm)
cd_11nm

{'Negative': [53599.0, 53, 256870.67924528304, 0.25313071133734516],
 'Positive': [41029.0, 50, 208427.31999999998, 0.20539267435563216],
 'Polar': [51940.0, 65, 202965.53846153847, 0.20001041488537255],
 'Hydrophobic': [69409.0, 86, 204998.67441860464, 0.20201394893047958],
 'Aromatic': [12257.0, 22, 141512.63636363635, 0.13945225049117052]}

### Calc. fractional contacts for each AA group type 

In [95]:
fcnt_rg11nm, prgrp_11nm, aa_matx_11nm = bavg_frac_cnt(5, prot_11nmPEG, n20PEG_11nm, dmax ,u11nm_n20PEG, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [96]:
fcnt_rg11nm

{'Negative': array([0.15511669, 0.1913301 , 0.25426301, 0.26247466, 0.24675896]),
 'Positive': array([0.124646  , 0.22803747, 0.29903196, 0.20447496, 0.16070977]),
 'Polar': array([0.31406271, 0.22449271, 0.18089477, 0.17145991, 0.18093769]),
 'Hydrophobic': array([0.21739405, 0.21806355, 0.18066591, 0.22677905, 0.20938313]),
 'Aromatic': array([0.18878055, 0.13807618, 0.08514436, 0.13481142, 0.20221044]),
 'total_frac': array([0.22587269, 0.21765914, 0.21560575, 0.30184805, 0.2587269 ])}

In [97]:
fc_11nm_mean = np.array([np.mean(fcnt_rg11nm['Negative']), np.mean(fcnt_rg11nm['Positive'])
                        ,np.mean(fcnt_rg11nm['Polar']),np.mean(fcnt_rg11nm['Hydrophobic'])
                        , np.mean(fcnt_rg11nm['Aromatic'])])
fc_11nm_mean

array([0.22198868, 0.20338003, 0.21436956, 0.21045714, 0.14980459])

In [98]:
fc_11nm_std = np.array([np.std(fcnt_rg11nm['Negative']), np.std(fcnt_rg11nm['Positive'])
                       ,np.std(fcnt_rg11nm['Polar']),np.std(fcnt_rg11nm['Hydrophobic'])
                       , np.std(fcnt_rg11nm['Aromatic'])])
fc_11nm_std

array([0.04172799, 0.05962932, 0.05314262, 0.01588115, 0.04197331])

In [99]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(x_pos, fc_09nm_mean, width, yerr=fc_09nm_std, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc_11nm_mean, width, yerr=fc_11nm_std, ecolor='black',capsize=5, color='c')
plt.title(r'Fractional Contacts Rg restrained', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 0.9 nm', 'Rg = 1.1 nm'], frameon=False)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20 ns blocks

In [100]:
np.mean(fcnt_rg11nm['total_frac'])

0.2439425051334702

In [101]:
np.std(fcnt_rg11nm['total_frac'])

0.03283880542374341

In [102]:
prgrp_11nm

{'Negative': array([array([2.042    , 2.7384733]), array([6.482     , 2.85055714]),
        array([8.149    , 3.3521335]), array([13.7675    ,  4.18693728]),
        array([20.3745    ,  5.00632098])], dtype=object),
 'Positive': array([array([2.434     , 3.64357572]), array([13.796     ,  4.46412186]),
        array([22.2895    ,  4.76294969]), array([29.9095   ,  7.0254046]),
        array([49.5605    ,  5.32356457])], dtype=object),
 'Polar': array([array([4.312     , 4.05544769]), array([8.042     , 4.19299845]),
        array([9.875     , 4.99453451]), array([11.6395    ,  4.93918412]),
        array([25.1535    ,  6.22124889])], dtype=object),
 'Hydrophobic': array([array([3.8665    , 4.28365238]), array([9.954     , 5.18988285]),
        array([9.8625    , 4.44854962]), array([17.444     ,  6.35073728]),
        array([30.4305    ,  6.65816564])], dtype=object),
 'Aromatic': array([array([1.107     , 2.06168645]), array([1.4465    , 2.01820161]),
        array([0.849     , 1.357

In [ ]:
# matrix containing the avg number of PLGA residues for each block for each amino acid 
np.where(aa_matx_15nm[0, 0] != 0)

### Avg no. PLGA residues per BSA AA residue group 

In [ ]:
prgrp_1_5nm

In [ ]:
mean_15nm = np.zeros(shape=5)
std_15nm = np.zeros(shape=5)
count = 0
for key, value in prgrp_1_5nm.items():
    mpl_15nm = []
    var_15nm = []
    for i in prgrp_1_5nm[str(key)].flat:
        mpl_15nm.append(i[0])
        var_15nm.append((i[1])**2)
    
    # calc frac cont averages
    mean_15nm[count] = np.mean(mpl_15nm)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_15nm[count] = np.std(mpl_15nm)
    #std_15nm[count] = np.sqrt(np.sum(var_15nm)/5)
    
    count += 1


In [ ]:
mean_15nm

In [ ]:
std_15nm

In [ ]:
x_pos = np.arange(5)
width = 0.35
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, mean_12nm, width, yerr=std_12nm, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, mean_15nm, width, yerr=std_15nm, ecolor='black',capsize=5, color='c')
plt.title(r'No. of PLGA residues Rg restrained', fontsize=15)
plt.xticks(x_pos+width/2, labels=aa_types, fontsize=12)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm'], frameon=False)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

### Protein/polymer contact map movie

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 1.5 nm res.", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_15nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('1.5nm_res.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()

# 1.4 nm restrained Rg PEG 100 ns trajectory

### Contact Analyis

In [139]:
# Set up the MD Simulation
u14nm_n20PEG = mda.Universe("1.4nmPEG_res/1.4nm_bsapegonly.pdb", "1.4nmPEG_res/nopbc_1.4pegbsa.xtc")

In [140]:
u14nm_n20PEG

<Universe with 9665 atoms>

In [141]:
pn20_len14nm = len(u14nm_n20PEG.trajectory)
pn20_len14nm

10001

In [142]:
#Select all the PLGA residues, heavy atoms only 
pn20_14nm = u14nm_n20PEG.select_atoms("resname sPEG PEG tPEG and not type H")
pn20_14nm

<AtomGroup with 183 atoms>

In [143]:
# Select BSA residues, heavy atoms only 
prot_14nm = u14nm_n20PEG.select_atoms("protein and not type H")
prot_14nm

<AtomGroup with 4653 atoms>

In [144]:
#dmax = 4.0, protein group(4653 atoms), plga atom group (543 atoms), took 381.6 s (6 min 36s on 4 cores)
start = 0
end = pn20_len14nm - 1
s_time = timeit.default_timer()
h2di_14nm = aa_frmcount(prot_14nm, pn20_14nm, dmax, u14nm_n20PEG, start, end)
timeit.default_timer() - s_time
#h2di_14nm

227.5563335912302

In [145]:
h2di_14nm

{'<Residue THR, 2>': [78.0, 0.0078],
 '<Residue HIS, 3>': [71.0, 0.0071],
 '<Residue LYS, 4>': [99.0, 0.0099],
 '<Residue SER, 5>': [68.0, 0.0068],
 '<Residue GLU, 6>': [86.0, 0.0086],
 '<Residue ALA, 8>': [3.0, 0.0003],
 '<Residue HIS, 9>': [235.0, 0.0235],
 '<Residue LYS, 12>': [688.0, 0.0688],
 '<Residue ASP, 13>': [1133.0, 0.1133],
 '<Residue LEU, 14>': [1544.0, 0.1544],
 '<Residue GLY, 15>': [583.0, 0.0583],
 '<Residue GLU, 16>': [180.0, 0.018],
 '<Residue GLU, 17>': [6751.0, 0.6751],
 '<Residue HIS, 18>': [6100.0, 0.61],
 '<Residue LYS, 20>': [53.0, 0.0053],
 '<Residue GLN, 33>': [357.0, 0.0357],
 '<Residue CYS, 34>': [143.0, 0.0143],
 '<Residue PRO, 35>': [330.0, 0.033],
 '<Residue ASP, 37>': [135.0, 0.0135],
 '<Residue GLU, 38>': [261.0, 0.0261],
 '<Residue VAL, 40>': [18.0, 0.0018],
 '<Residue LYS, 41>': [92.0, 0.0092],
 '<Residue ASN, 44>': [55.0, 0.0055],
 '<Residue GLU, 45>': [2.0, 0.0002],
 '<Residue GLU, 48>': [334.0, 0.0334],
 '<Residue LYS, 51>': [532.0, 0.0532],
 '<Res

In [146]:
len(h2di_14nm.keys())

205

In [147]:
pr_res14nm = list(prot_14nm.residues)
ss_res14nm = [str(row) for row in pr_res14nm]
rkg_14nm = {key:h2di_14nm[key][0] for key, value in h2di_14nm.items()}
plg_14nmaa = pd.DataFrame(data=ss_res14nm, columns=["BSA_des_res"])
plg_14nmaa['mda_plga_frm_1.4nm'] = plg_14nmaa['BSA_des_res'].map(rkg_14nm)
plg_14nmaa['BSA_des_res'] = red_bsa
plg_14nmaa['mda_plga_frm_1.4nm'] = plg_14nmaa['mda_plga_frm_1.4nm'].replace('nan', np.nan).fillna(0)
plg_14nmaa.head()

,BSA_des_res,mda_plga_frm_1.4nm
0,ASP 1,0.0
1,THR 2,78.0
2,HIS 3,71.0
3,LYS 4,99.0
4,SER 5,68.0


In [148]:
# Extract mean occupancy values
pr_res_14ur = list(prot_14nm.residues)
ss_res_14ur = [str(row) for row in pr_res_14ur]
rkg_14ur = {key:h2di_14nm[key][1] for key, value in h2di_14nm.items()}
plg_09nm_occ['mda_occ_1.4nm'] = plg_09nm_occ['BSA_des_res'].map(rkg_14ur)
plg_09nm_occ['mda_occ_1.4nm'] = plg_09nm_occ['mda_occ_1.4nm'].replace('nan', np.nan).fillna(0)
plg_09nm_occ['mda_occ_1.4nm'] = plg_09nm_occ['mda_occ_1.4nm'].round(2)
plg_09nm_occ

,BSA_des_res,mda_occ_0.9nm,mda_occ_1.1nm,mda_occ_1.4nm
0,"<Residue ASP, 1>",0.00,0.00,0.00
1,"<Residue THR, 2>",0.00,0.00,0.01
2,"<Residue HIS, 3>",0.00,0.00,0.01
3,"<Residue LYS, 4>",0.00,0.00,0.01
4,"<Residue SER, 5>",0.00,0.00,0.01
...,...,...,...,...
578,"<Residue GLN, 579>",0.01,0.12,0.00
579,"<Residue THR, 580>",0.07,0.24,0.00
580,"<Residue ALA, 581>",0.02,0.14,0.00
581,"<Residue LEU, 582>",0.00,0.05,0.00


In [149]:
bsa_r = np.array(list(prot_14nm.resids)) # shape is 4652
m_occ_14nm = np.array(list(plg_09nm_occ['mda_occ_1.4nm'])) # shape is 583
m_occ_14New = np.zeros(shape=(4653))
at_ind = np.where(bsa_r[:-1] != bsa_r[1:])[0]
at_in_nw = np.sort(np.append([0,4653],at_ind))
nw_v = 0
for i in range(583):
    b = at_in_nw[i+1] +1
    m_occ_14New[nw_v:b] = m_occ_14nm[i]
    nw_v = at_in_nw[i+1] + 1 

In [150]:
np.nonzero(m_occ_14New)

(array([   8,    9,   10,   11,   12,   13,   14,   15,   16,   17,   18,
          19,   20,   21,   22,   23,   24,   25,   26,   27,   28,   29,
          30,   31,   32,   33,   34,   35,   36,   37,   38,   39,   40,
          41,   42,   43,   44,   45,   46,   47,   48,   62,   63,   64,
          65,   66,   67,   68,   69,   70,   71,   94,   95,   96,   97,
          98,   99,  100,  101,  102,  103,  104,  105,  106,  107,  108,
         109,  110,  111,  112,  113,  114,  115,  116,  117,  118,  119,
         120,  121,  122,  123,  124,  125,  126,  127,  128,  129,  130,
         131,  132,  133,  134,  135,  136,  137,  138,  139,  140,  141,
         142,  143,  144,  145,  146,  147,  148,  149,  150,  162,  163,
         164,  165,  166,  167,  168,  169,  170,  266,  267,  268,  269,
         270,  271,  272,  273,  274,  275,  276,  277,  278,  279,  280,
         281,  282,  283,  284,  285,  286,  287,  299,  300,  301,  302,
         303,  304,  305,  306,  307, 

### Visualize Occupanct on protein 

In [151]:
prot_14nm.occupancies = m_occ_14New
prot_14nm.occupancies

array([0., 0., 0., ..., 0., 0., 0.])

In [152]:
with mda.Writer("prot_14nmRes.pdb") as pdb:
    pdb.write(prot_14nm)

### Residue Importance: 1.4 nm restrained 

In [153]:
# Need to fix function, the residue number are not counting the other 2 PLGA oligomers cuz of same resid number
trjmap_14nmPEG = prot_poly_cntmovie(prot_14nm, pn20_14nm, dmax, u14nm_n20PEG, 0, 10000)
#trj_ppmap_12nm_chC = prot_poly_cntmovie(prot, all_pn20_C, dmax, u_pn20, 0, 10000)

In [154]:
np.save('1.4nm_res.npy', trjmap_14nmPEG)    # .npy extension is added if not given

In [ ]:
trj_ppmap_2nm = np.load('2nm_res.npy', allow_pickle=True)
trj_ppmap_2nm

In [155]:
trjmap_14nmPEG[0].shape

(583, 60)

In [156]:
np.sum(trjmap_14nmPEG[1000][0])

0.0

In [157]:
kj = np.zeros(shape=(10000, 583))
kj[:,582].shape

(10000,)

In [158]:
pp_14nm_ct = np.zeros(shape=(10000, 583))
for i in range(10000):
    for j in range(583):
        pp_14nm_ct[i][j] = np.sum(trjmap_14nmPEG[i][j])

In [159]:
pp_14nmtot = np.zeros(shape=(583))
for i in range(583):
    pp_14nmtot[i] = np.sum(pp_14nm_ct[:,i])
#pp_12nmtot

In [160]:
np.nonzero(pp_14nmtot)

(array([  1,   2,   3,   4,   5,   7,   8,  11,  12,  13,  14,  15,  16,
         17,  19,  32,  33,  34,  36,  37,  39,  40,  43,  44,  47,  50,
         51,  53,  54,  55,  56,  57,  58,  74,  75,  76,  77,  78,  79,
         80,  81,  82,  83,  84,  85,  87,  88,  90,  91,  92,  93,  96,
        110, 111, 112, 114, 125, 126, 127, 128, 129, 130, 133, 154, 156,
        157, 158, 160, 161, 164, 166, 174, 175, 178, 179, 223, 256, 260,
        263, 264, 265, 266, 267, 268, 270, 271, 272, 274, 275, 277, 278,
        279, 280, 281, 282, 283, 284, 293, 294, 295, 296, 297, 298, 299,
        300, 301, 302, 304, 305, 306, 309, 310, 311, 312, 315, 338, 346,
        349, 350, 353, 354, 357, 358, 361, 362, 363, 364, 365, 366, 367,
        368, 369, 370, 371, 373, 374, 376, 377, 378, 380, 381, 384, 387,
        388, 391, 392, 393, 394, 395, 396, 397, 408, 439, 442, 473, 477,
        481, 484, 487, 488, 490, 493, 499, 500, 501, 502, 503, 505, 506,
        507, 508, 510, 516, 538, 539, 540, 541, 543

In [185]:
a_peg = np.sum(pp_14nmtot)

In [186]:
b_peg = np.sum(pp_11nmtot)

In [187]:
c_peg = np.sum(pp_09nmtot)

In [184]:
plt.close('all')

In [191]:
y_pos = np.arange(583) + 1
wid = np.zeros(shape=583)
wid += 1.5
#wid
fig = plt.figure(figsize=(12,12))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(y_pos, pp_09nmtot/c_peg, align='center',width=wid, alpha=0.5, color='#1D77CF',label='0.9 nm PEG')
plt.bar(y_pos+0.25, pp_11nmtot/b_peg, align='center',width=wid, color='#562A8B', alpha=0.3, label='1.1 nm PEG')
plt.bar(y_pos+0.3, pp_14nmtot/a_peg, align='center',width=wid, color='#4E4C4D', alpha=0.3, label='1.4 nm PEG')
#plt.bar(y_pos+0.25, pp_11nmtot/b_peg, align='center',width=wid, color='#562A8B', alpha=0.3, label='1.1 nm PEG')
#plt.bar(y_pos, pp_09nmtot/c_peg, align='center',width=wid, alpha=0.5, color='#1D77CF',label='0.9 nm PEG')
plt.title("BSA in water with PEG restrained, 100 ns", fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,600])
plt.ylim([0,0.2])
plt.legend(fontsize=14)
plt.ylabel(r'Normalized Total No. of PEG contacts', fontsize=15)
plt.xlabel(r'BSA Residue ID', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'BSA Residue ID')

### Total number of residues that are within 4 angstroms of a PLGA oligomer residue within a 100 ns trajectory block

In [162]:
apl_14nm = []

# Some residues don't have any contact with the 3 N = 20 PLGA oligomers within 100 ns,
# Put residues that do have contact with BSA in a separate list
for index, r_pl in plg_14nmaa.iterrows():
    if r_pl['mda_plga_frm_1.4nm'] != 0:
        apl_14nm.append(r_pl['BSA_des_res'])
        
# This chunk of code gets an AA count from the above list, in order 
# to get a total number of residues that contact BSA
cpl_14nm = []

for index, r_a in aa_count.iterrows():
    count = 0
    for i in range(len(apl_14nm)):
        if r_a['Amino_acids'] in apl_14nm[i]:
            count += 1
    cpl_14nm.append(count)      
        
aa_count['peg_1.4nm_100ns'] = cpl_14nm
#aa_count.drop('No_of_surf_res (VMD)', axis=1, inplace=True)
aa_count

,Amino_acids,No_of_surf_res (MDAnalysis),No_of_surf_res (VMD),peg_0.9nm_100ns,peg_1.1nm_100ns,peg_1.4nm_100ns
0,LYS,59,59,30,34,33
1,ARG,23,23,6,9,3
2,HIS,16,16,4,7,7
3,ASP,40,40,20,22,24
4,GLU,59,59,25,31,29
5,SER,23,23,6,10,8
6,THR,29,29,18,21,13
7,CYS,24,24,12,12,10
8,TYR,19,19,3,7,3
9,ASN,14,14,3,4,8


In [163]:
# This gives the total number of residues that are within 4 angstroms of a PLGA oligomer residue
# within a 100 ns trajectory block
aa_count['peg_1.4nm_100ns'].sum()

205

In [164]:
# This gives the total number of residues that are within 4 angstroms of a water molecule
# within a 1 ns trajectory block
aa_count['No_of_surf_res (MDAnalysis)'].sum()

487

In [165]:
# This gives the total fraction of contacts within the 1.2 nm Rg 100 ns trajectory
aa_count['peg_1.4nm_100ns'].sum()/aa_count['No_of_surf_res (MDAnalysis)'].sum()

0.4209445585215606

In [167]:
# Mean occupancy and std deviation 
ll_mo2nm = [value[1] for key, value in h2di_14nm.items()]
print("Mean Occpancy (1.4 nm Rg): "+str(np.mean(ll_mo2nm)), "Occ. std. dev.: "+str(np.std(ll_mo2nm)))

Mean Occpancy (1.4 nm Rg): 0.07033170731707317 Occ. std. dev.: 0.1632142219996123


In [ ]:
cd_2nm = frac_cont(h2di_2nm)
cd_2nm

### Calc. fractional contacts for each AA group type 

In [171]:
fcnt_rg14nm, prgrp_14nm, aa_matx_14nm = bavg_frac_cnt(5, prot_14nm, pn20_14nm, dmax, u14nm_n20PEG, no_surf, 0, 10000)

0
2000
2000
4000
4000
6000
6000
8000
8000
10000


In [172]:
fcnt_rg14nm

{'Negative': array([0.23001977, 0.17563879, 0.18279515, 0.21668994, 0.2017111 ]),
 'Positive': array([0.34159842, 0.16158551, 0.21958239, 0.19811406, 0.17486378]),
 'Polar': array([0.1897769 , 0.25689744, 0.21626921, 0.22442349, 0.17954485]),
 'Hydrophobic': array([0.18819543, 0.13387627, 0.14250035, 0.14964026, 0.16276217]),
 'Aromatic': array([0.05040949, 0.272002  , 0.2388529 , 0.21113225, 0.28111811]),
 'total_frac': array([0.10677618, 0.19712526, 0.09445585, 0.09856263, 0.26078029])}

In [173]:
fc_14nm_mean = np.array([np.mean(fcnt_rg14nm['Negative']), np.mean(fcnt_rg14nm['Positive'])
                        ,np.mean(fcnt_rg14nm['Polar']),np.mean(fcnt_rg14nm['Hydrophobic'])
                        , np.mean(fcnt_rg14nm['Aromatic'])])
fc_14nm_mean

array([0.20137095, 0.21914883, 0.21338238, 0.15539489, 0.21070295])

In [174]:
fc_14nm_std = np.array([np.std(fcnt_rg14nm['Negative']), np.std(fcnt_rg14nm['Positive'])
                       ,np.std(fcnt_rg14nm['Polar']),np.std(fcnt_rg14nm['Hydrophobic'])
                       , np.std(fcnt_rg14nm['Aromatic'])])
fc_14nm_std

array([0.02031131, 0.06436032, 0.02729834, 0.01893456, 0.08390765])

In [175]:
x_pos = np.arange(5)
width = 0.28
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
fig = plt.figure(figsize=(7,7))
fig.canvas.layout.width = '800px'
fig.canvas.layout.height = '700px'
plt.bar(x_pos, fc_09nm_mean, width, yerr=fc_09nm_std, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, fc_11nm_mean, width, yerr=fc_11nm_std, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), fc_14nm_mean, width, yerr=fc_14nm_std, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'Fractional Contacts Rg restrained', fontsize=15)
plt.xticks(x_pos+width, labels=aa_types, fontsize=12)
plt.ylim(0,0.4)
plt.legend(['Rg = 0.9 nm', 'Rg = 1.1 nm', 'Rg = 1.4 nm'], frameon=False)
plt.ylabel(r'Fractional Contacts', fontsize=15)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Fractional Contacts')

### Total fraction of contacts: averages and std dev calc from 5 20 ns blocks

In [176]:
np.mean(fcnt_rg14nm['total_frac'])

0.1515400410677618

In [177]:
np.std(fcnt_rg14nm['total_frac'])

0.06645367919891182

In [ ]:
prgrp_2nm

In [ ]:
# matrix containing the avg number of PLGA residues for each block for each amino acid 
np.where(aa_matx_2nm[0] != 0)

### Avg no. PLGA residues per BSA AA residue group 

In [ ]:
prgrp_2nm

In [ ]:
mean_2nm = np.zeros(shape=5)
std_2nm = np.zeros(shape=5)
count = 0
for key, value in prgrp_2nm.items():
    mpl_2nm = []
    var_2nm = []
    for i in prgrp_2nm[str(key)].flat:
        mpl_2nm.append(i[0])
        var_2nm.append((i[1])**2)
    
    # calc frac cont averages
    mean_2nm[count] = np.mean(mpl_2nm)
    
    # calc frac cont std dev: https://stats.stackexchange.com/questions/25848/how-to-sum-a-standard-deviation 
    std_2nm[count] = np.std(mpl_2nm)
    #std_2nm[count] = np.sqrt(np.sum(var_2nm)/5)
    
    count += 1


In [ ]:
mean_2nm

In [ ]:
std_2nm

In [ ]:
mean_15nm

In [ ]:
std_15nm

In [ ]:
x_pos = np.arange(5)
width = 0.28
aa_types = ["Negative", "Positive", "Polar", "Hydrophobic", "Aromatic"]
plt.figure(figsize=(7,7))
plt.bar(x_pos, mean_12nm, width, yerr=std_12nm, ecolor='black',capsize=5, color='royalblue')
plt.bar(x_pos+width, mean_15nm, width, yerr=std_15nm, ecolor='black',capsize=5, color='c')
plt.bar(x_pos+(2*width), mean_2nm, width, yerr=std_2nm, ecolor='black',capsize=5, color='lightslategray')
plt.title(r'No. of PLGA residues Rg restrained', fontsize=15)
plt.xticks(x_pos+width, labels=aa_types, fontsize=12)
plt.legend(['Rg = 1.2 nm', 'Rg = 1.5 nm', 'Rg = 2 nm'], frameon=False)
plt.ylabel(r'No. of PLGA residues', fontsize=15)

### Protein/polymer contact map movie

In [ ]:
np.where(trj_load2nm_res[9920] == 0 )

In [ ]:
trj_load2nm_res[9920]

In [ ]:
trj_load2nm_res.shape

In [ ]:
fig = plt.figure(figsize=(10,10))

# Set the axis and the plot titles pp

plt.title("BSA/PLGA contact map 2 nm restrained", fontsize=22, loc='left')
plt.xlabel("PLGA Residue No.", fontsize=22)
plt.ylabel("BSA Residue No.", fontsize=20)

 # Set the axis range 
plt.ylim(583, 0)
plt.xlim(0, 60)

# Plot bands for each chain 
BANDS = (
    (0, 20, "purple", "B"),
    (20, 40, "blue", "C"),
    (40, 60, "green", "D"),
)
    
text_y = 0.98 # Close to the top
for start, stop, color, band in BANDS:
    plt.axvspan(start, stop,color=color, alpha=0.15)
    text_x = middle_of_band(start,stop)
    plt.text(
        text_x,
        text_y,
        "PLGA chain " + band,
        color=color,
        fontsize=18,
        transform=fig.gca().transAxes,
        horizontalalignment='center',
        verticalalignment='center',
        style='italic',
    )
    
plt.text(0.94, 1, "Time [ns]:", fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')

# Set tick label size
fig.gca().tick_params(axis='both', which='major', labelsize=20)

ims = []
for i in range(10000):
    data = trj_ppmap_2nm[i]
    im = plt.imshow(data, aspect='auto', cmap='Greys')
    t_sim = plt.text(1.03, 1, str(i/100), fontsize=20, transform=fig.gca().transAxes, horizontalalignment='right', verticalalignment='bottom')
    ims.append([im, t_sim])
    
ani = animation.ArtistAnimation(fig, ims, blit=True, repeat=False)
ani.save('2nm_res.mp4', writer='ffmpeg', fps=50, bitrate=100000)
#plt.tight_layout()
#plt.show()